# AE for Image Denoising

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load the MNIST dataset
(train_images, _), (test_images, _) = mnist.load_data()

# Preprocess the data and introduce noise
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Add random noise to the images
noise_factor = 0.5

train_images_noisy =
train_images + noise_factor * np.random.normal(loc=0.0, scale=1.0,
                                                                    size=train_images.shape)
test_images_noisy =
test_images + noise_factor * np.random.normal(loc=0.0, scale=1.0,
                                                                  size=test_images.shape)

# Clip the pixel values to [0, 1]
train_images_noisy = np.clip(train_images_noisy, 0., 1.)
test_images_noisy = np.clip(test_images_noisy, 0., 1.)

# Flatten the images for the autoencoder
train_images = train_images.reshape((len(train_images),
                                     np.prod(train_images.shape[1:])))
test_images = test_images.reshape((len(test_images),
                                   np.prod(test_images.shape[1:])))
train_images_noisy = train_images_noisy.reshape((len(train_images_noisy),
                                                 np.prod(train_images_noisy.shape[1:])))
test_images_noisy = test_images_noisy.reshape((len(test_images_noisy),
                                               np.prod(test_images_noisy.shape[1:])))

# Define the autoencoder architecture
encoding_dim = 32

input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

# Create the autoencoder model
autoencoder = Model(input_img, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
early_stopping = EarlyStopping(monitor='val_loss', patience=3,
                               restore_best_weights=True)
history = autoencoder.fit(train_images_noisy, train_images,
                          epochs=50,
                          batch_size=128,
                          shuffle=True,
                          validation_data=(test_images_noisy, test_images),
                          callbacks=[early_stopping])

# Encoder model
encoder = Model(input_img, encoded)

# Decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Visualize the encoder design, hidden state, and decoder
encoded_imgs = encoder.predict(test_images_noisy)
decoded_imgs = decoder.predict(encoded_imgs)



In [ ]:
# Display a few test images and their denoised counterparts
n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original images
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(test_images_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display hidden state (encoded representation)
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(8, 4), cmap='gray')  # Adjust reshape dimensions accordingly
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstructed images
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()
